In [56]:
# Python program to read
# json file
 
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from ipywidgets import interact

str_list,start_list = []
for i in range(0,95,5):
    str_list.append(f"{i}-{i+5}")
    start_list.append(i)
for i in range(100,310,10):
    str_list.append(f"{i}-{i+10}") 
    start_list.append(i)

eloise_prop_burn = [25.10310991, 8.89068476, 6.798758934, 5.752796021, 5.229814565, 4.706833108, 4.183851652,
                   3.660870195, 3.660870195, 3.137888739, 3.137888739, 2.614907282, 2.091925826, 0, 0.114292392,
                   0.342877175, 0.342877175, 0.400023371, 0.742900546, 0.914339134, 0.914339134, 1.142923917,
                   1.371508701, 1.885824463]
eloise_prop_burn = np.concatenate((eloise_prop_burn,np.zeros(6))) 

ross_prop_mass = [4.1, 2.3, 2.4, 2.6, 2.7, 2.9, 3.5, 4.0, 3.2, 2.3, 2.5, 2.7, 3.0, 3.3, 3.9, 4.4, 5.0, 6.9, 17.3]
ross_mass_reversed = ross_prop_mass[::-1]
ross_mass_reversed = np.concatenate((ross_mass_reversed,np.zeros((len(str_list)-len(ross_mass_reversed)))))

dir_path = r'src/SpaceX_*.json'
files = glob.glob(dir_path)

t_meco = 120+36
alt_meco = 67.6
t_sei_seco = (8*60+54) - (120+48)
alt_sei = 79
alt_seco = 216
prop_burned_total = np.zeros((len(str_list),len(files))) 

In [62]:
for count, filename in enumerate(files):
    prop_s1 = 418700
    prop_s2 = 111500
    burnrate_s1 = prop_s1/t_meco
    burnrate_s2 = prop_s2/t_sei_seco

    time, time_bins_s1, time_bins_s2 = [], [], []
    time_elapsed_s1, time_elapsed_s2 = [], []
    vel = []
    alt, alt_bins = [], []

    # Opening JSON file
    with open(filename, 'r') as file:
        for line in file:
            time.append(json.loads(line)["time"])
            vel.append(json.loads(line)["velocity"])
            alt.append(json.loads(line)["altitude"])
        
    alt_bins_s1 = np.arange(0,alt_meco,5)
    alt_bins_s1 = np.append(alt_bins_s1,alt_meco)
    q,mod = divmod(alt_sei,5)
    alt_bins_s2 = np.arange(q*5,95,5)
    q,mod = divmod(alt_seco,10)
    alt_bins_s2 = np.concatenate((alt_bins_s2,np.arange(100,alt_seco,10),[10*(q+1)]))
    for i in range(len(alt_bins_s1)):
        idx = (np.abs(alt - alt_bins_s1[i])).argmin()
        time_bins_s1.append(time[idx])
    for i in range(len(alt_bins_s2)):
        idx = (np.abs(alt - alt_bins_s2[i])).argmin()
        time_bins_s2.append(time[idx])

    dist_trav = []   
    time_elapsed = []     
    for i in range(1,len(alt_bins_s1)):
        dist_trav.append(alt_bins_s1[i]-alt_bins_s1[i-1])
        time_elapsed_s1.append(time_bins_s1[i]-time_bins_s1[i-1])
    for i in range(1,len(alt_bins_s2)):
        dist_trav.append(alt_bins_s2[i]-alt_bins_s2[i-1])
        time_elapsed_s2.append(time_bins_s2[i]-time_bins_s2[i-1])

    prop_burned_s1 = np.array(time_elapsed_s1) * burnrate_s1 / (prop_s1+prop_s2) * 100
    prop_burned_s2 = np.array(time_elapsed_s2) * burnrate_s2 / (prop_s1+prop_s2) * 100
    number_zeros, _ = divmod(alt_bins_s2[0] - alt_bins_s1[-1],5)

    prop_burned_total[:len(prop_burned_s1),count] += prop_burned_s1
    start = (np.abs(int(str_list.split("-")[0]) - alt_bins_s2[0])).argmin()
    print(start)
    print(str_list[start],alt_bins_s2[0])
    start = 0
    stop = 15
    prop_burned_total[start:stop,count] = prop_burned_s2[:-1:] 
 

launch_pairs = []
for i in range(0,len(files)):
    pair = (f"{filename[11:19]}",i)
    launch_pairs.append(pair)

def plot_layer(launch):
    df_bar = pd.DataFrame(dict(graph=str_list, m=prop_burned_total[:,launch], n=ross_mass_reversed)) 
    ind = np.arange(len(df_bar))
    width = 0.4
    fig, ax = plt.subplots(figsize=(10,10))
    ax.barh(ind + width, df_bar.m, width, label='Auto Propellant Distribution')
    ax.barh(ind + width*2, df_bar.n, width, label='Ross2014')
    ax.set_xlabel("Propellant Burned / %")
    ax.set_ylabel("Altitude / km")
    ax.set_xlim(0,30)
    ax.set_title(f"Vertical Propellant Distribution - Starlink Launch {filename[11:19]}")
    ax.legend()
    ax.set(yticks=ind, yticklabels=df_bar.graph, ylim=[2*width - 1, len(df_bar)]);

interact(plot_layer, launch=launch_pairs);

AttributeError: 'list' object has no attribute 'split'